# Import the library

In [ ]:
import os

import torch
from trainer import Trainer, TrainerArgs

# from TTS.bin.compute_embeddings import compute_embeddings
from compute_embeddings import compute_embeddings # use custom formatter without forking the lib
from TTS.bin.resample import resample_files
from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.configs.vits_config import VitsConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.vits import CharactersConfig, Vits, VitsArgs, VitsAudioConfig
# from TTS.utils.downloaders import download_vctk
# from TTS.config import load_config
# from TTS.config.shared_configs import BaseDatasetConfig
# from TTS.tts.datasets import load_tts_samples
# from TTS.tts.utils.managers import save_file
# from TTS.tts.utils.speakers import SpeakerManager
from TTS.tts.datasets.formatters import vctk
from functools import partial

from tqdm import tqdm

torch.set_num_threads(24)

# Setup constants

In [2]:
# Current path
CURRENT_PATH = os.getcwd()

# Name of the run for the Trainer
RUN_NAME = "KhongKhunTTS-TH-VCTK"

# Path where you want to save the models outputs (configs, checkpoints and tensorboard logs)
OUT_PATH = os.getcwd()

# If you want to do transfer learning and speedup your training you can set here the path to the model
RESTORE_PATH = None

# This paramter is useful to debug, it skips the training epochs and just do the evaluation  and produce the test sentences
SKIP_TRAIN_EPOCH = False

# Set here the batch size to be used in training and evaluation
BATCH_SIZE = 32

# Training Sampling rate and the target sampling rate for resampling the downloaded dataset (Note: If you change this you might need to redownload the dataset !!)
# Note: If you add new datasets, please make sure that the dataset sampling rate and this parameter are matching, otherwise resample your audios
SAMPLE_RATE = 32000

# Max audio length in seconds to be used in training (every audio bigger than it will be ignored)
MAX_AUDIO_LEN_IN_SECONDS = 10

### Download VCTK dataset
VCTK_DOWNLOAD_PATH = os.path.join(CURRENT_PATH, "commonvoice-to-vctk")
# Define the number of threads used during the audio resampling
NUM_RESAMPLE_THREADS = 10

# Dataset configuration

In [3]:
# Check if VCTK dataset is not already downloaded, if not download it
# if not os.path.exists(VCTK_DOWNLOAD_PATH):
#     print(">>> Downloading VCTK dataset:")
#     download_vctk(VCTK_DOWNLOAD_PATH, True)
    # resample_files(VCTK_DOWNLOAD_PATH, SAMPLE_RATE, file_ext="flac", n_jobs=NUM_RESAMPLE_THREADS)

# init configs
vctk_config = BaseDatasetConfig(
    formatter="vctk_32k",
    dataset_name="vctk",
    meta_file_train="",
    meta_file_val="",
    path=VCTK_DOWNLOAD_PATH,
    language="th",
    ignored_speakers=[
        "cv017", # Female Teenager
        "cv048", # Female Teenager
        "cv039", # Female Adult
        "cv052", # Female Adult
        "cv069", # Male Teenager
        "cv054", # Male Teenager
        "cv049", # Male Adult
        "cv026", # Male Adult
    ], # For testing set
)

# Add here all datasets configs, in our case we just want to train with the VCTK dataset then we need to add just VCTK. Note: If you want to add new datasets, just add them here and it will automatically compute the speaker embeddings (d-vectors) for this new dataset :)
DATASETS_CONFIG_LIST = [vctk_config]

# Extract speaker embeddings

In [4]:
# from tqdm import tqdm

# from TTS.config import load_config
# from TTS.config.shared_configs import BaseDatasetConfig
# from TTS.tts.datasets import load_tts_samples
# from TTS.tts.utils.managers import save_file
# from TTS.tts.utils.speakers import SpeakerManager
# from TTS.tts.datasets.formatters import vctk
# from functools import partial

# def compute_embeddings(
#     model_path,
#     config_path,
#     output_path,
#     formatter_name=None,
#     dataset_name=None,
#     dataset_path=None,
#     meta_file_train=None,
#     meta_file_val=None,
# ):
#     use_cuda = torch.cuda.is_available()

#     c_dataset = BaseDatasetConfig()
#     c_dataset.formatter = formatter_name
#     c_dataset.dataset_name = dataset_name
#     c_dataset.path = dataset_path
#     if meta_file_train is not None:
#         c_dataset.meta_file_train = meta_file_train
#     if meta_file_val is not None:
#         c_dataset.meta_file_val = meta_file_val
#     meta_data_train, meta_data_eval = load_tts_samples(c_dataset, eval_split=True, formatter=custom_vctk)
    
#     samples = meta_data_train + meta_data_eval

#     encoder_manager = SpeakerManager(
#         encoder_model_path=model_path,
#         encoder_config_path=config_path,
#         d_vectors_file_path=None,
#         use_cuda=use_cuda,
#     )

#     class_name_key = encoder_manager.encoder_config.class_name_key

#     # compute speaker embeddings
#     speaker_mapping = {}

#     for fields in tqdm(samples):
#         class_name = fields[class_name_key]
#         audio_file = fields["audio_file"]
#         embedding_key = fields["audio_unique_name"]

#         # Only update the speaker name when the embedding is already in the old file.
#         if embedding_key in speaker_mapping:
#             speaker_mapping[embedding_key]["name"] = class_name
#             continue

#         embedd = encoder_manager.compute_embedding_from_clip(audio_file)

#         # create speaker_mapping if target dataset is defined
#         speaker_mapping[embedding_key] = {}
#         speaker_mapping[embedding_key]["name"] = class_name
#         speaker_mapping[embedding_key]["embedding"] = embedd

#     if speaker_mapping:
#         # save speaker_mapping if target dataset is defined
#         if os.path.isdir(output_path):
#             mapping_file_path = os.path.join(output_path, "speakers.pth")
#         else:
#             mapping_file_path = output_path

#         if os.path.dirname(mapping_file_path) != "":
#             os.makedirs(os.path.dirname(mapping_file_path), exist_ok=True)

#         save_file(speaker_mapping, mapping_file_path)
#         print("Speaker embeddings saved at:", mapping_file_path)

In [5]:
SPEAKER_ENCODER_CHECKPOINT_PATH = (
    "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/model_se.pth.tar"
)
SPEAKER_ENCODER_CONFIG_PATH = "https://github.com/coqui-ai/TTS/releases/download/speaker_encoder_model/config_se.json"

D_VECTOR_FILES = []  # List of speaker embeddings/d-vectors to be used during the training

vctk_32k = partial(
    vctk,
    wavs_path="wav32_silence_trimmed",
)

# Iterates all the dataset configs checking if the speakers embeddings are already computated, if not compute it
for dataset_conf in DATASETS_CONFIG_LIST:
    # Check if the embeddings weren't already computed, if not compute it
    embeddings_file = os.path.join(vctk_config.path, "speakers.pth")
    if not os.path.isfile(embeddings_file):
        print(f">>> Computing the speaker embeddings for the {vctk_config.dataset_name} dataset")
        compute_embeddings(
            SPEAKER_ENCODER_CHECKPOINT_PATH,
            SPEAKER_ENCODER_CONFIG_PATH,
            embeddings_file,
            formatter_name=vctk_config.formatter,
            formatter=vctk_32k if vctk_config.formatter == "vctk_32k" else None,
            dataset_name=vctk_config.dataset_name,
            dataset_path=vctk_config.path,
            meta_file_train=vctk_config.meta_file_train,
            meta_file_val=vctk_config.meta_file_val,
        )

        # meta_data_train, meta_data_eval = load_tts_samples(vctk_config, eval_split=True, formatter=custom_vctk)

        # samples = meta_data_train + meta_data_eval

        # encoder_manager = SpeakerManager(
        #     encoder_model_path=SPEAKER_ENCODER_CHECKPOINT_PATH,
        #     encoder_config_path=SPEAKER_ENCODER_CONFIG_PATH,
        #     d_vectors_file_path=None,
        #     use_cuda=torch.cuda.is_available(),
        # )

        # class_name_key = encoder_manager.encoder_config.class_name_key

        # # compute speaker embeddings
        # speaker_mapping = {}

        # for fields in tqdm(samples):
        #     class_name = fields[class_name_key]
        #     audio_file = fields["audio_file"]
        #     embedding_key = fields["audio_unique_name"]

        #     # Only update the speaker name when the embedding is already in the old file.
        #     if embedding_key in speaker_mapping:
        #         speaker_mapping[embedding_key]["name"] = class_name
        #         continue

        #     embedd = encoder_manager.compute_embedding_from_clip(audio_file)

        #     # create speaker_mapping if target dataset is defined
        #     speaker_mapping[embedding_key] = {}
        #     speaker_mapping[embedding_key]["name"] = class_name
        #     speaker_mapping[embedding_key]["embedding"] = embedd
        
        # if speaker_mapping:
        #     # save speaker_mapping if target dataset is defined
        #     if os.path.isdir(embeddings_file):
        #         mapping_file_path = os.path.join(embeddings_file, "speakers.pth")
        #     else:
        #         mapping_file_path = embeddings_file

        #     if os.path.dirname(mapping_file_path) != "":
        #         os.makedirs(os.path.dirname(mapping_file_path), exist_ok=True)

        #     save_file(speaker_mapping, mapping_file_path)
        #     print("Speaker embeddings saved at:", mapping_file_path)

    D_VECTOR_FILES.append(embeddings_file)

# Audio config used in training.

In [6]:
audio_config = VitsAudioConfig(
    sample_rate=SAMPLE_RATE,
    hop_length=256,
    win_length=1024,
    fft_size=1024,
    mel_fmin=0.0,
    mel_fmax=None,
    num_mels=80,
)

# Model configuration

In [7]:
# Init VITSArgs setting the arguments that are needed for the KhongKhunTTS model
model_args = VitsArgs(
    d_vector_file=D_VECTOR_FILES,
    use_d_vector_file=True,
    d_vector_dim=512,
    num_layers_text_encoder=10,
    speaker_encoder_model_path=SPEAKER_ENCODER_CHECKPOINT_PATH,
    speaker_encoder_config_path=SPEAKER_ENCODER_CONFIG_PATH,
    resblock_type_decoder="2",  # In the YourTTS paper, trained using ResNet blocks type 2, if you like you can use the ResNet blocks type 1 like the VITS model
    # Useful parameters to enable the Speaker Consistency Loss (SCL) described in the paper
    # use_speaker_encoder_as_loss=True,
    # Useful parameters to enable multilingual training
    # use_language_embedding=True,
    # embedded_language_dim=4,
)

In [ ]:
# General training config, here you can change the batch size and others useful parameters
config = VitsConfig(
    output_path=OUT_PATH,
    model_args=model_args,
    run_name=RUN_NAME,
    project_name="KhongKhunTTS",
    run_description="""
            - KhongKhunTTS trained using CommonVoiceTH (VCTK structure)
        """,
    dashboard_logger="tensorboard",
    logger_uri=None,
    audio=audio_config,
    batch_size=BATCH_SIZE,
    batch_group_size=48,
    eval_batch_size=BATCH_SIZE,
    num_loader_workers=8,
    eval_split_max_size=256,
    print_step=50,
    plot_step=100,
    log_model_step=1000,
    save_step=5000,
    save_n_checkpoints=2,
    save_checkpoints=True,
    target_loss="loss_1",
    print_eval=False,
    use_phonemes=False,
    phonemizer="espeak",
    phoneme_language="en",
    compute_input_seq_cache=True,
    add_blank=True,
    text_cleaner="multilingual_cleaners",
    characters=CharactersConfig(
        characters_class="TTS.tts.models.vits.VitsCharacters",
        pad="_",
        eos="&",
        bos="*",
        blank=None,
        characters="ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz\u00af\u00b7\u00df\u00e0\u00e1\u00e2\u00e3\u00e4\u00e6\u00e7\u00e8\u00e9\u00ea\u00eb\u00ec\u00ed\u00ee\u00ef\u00f1\u00f2\u00f3\u00f4\u00f5\u00f6\u00f9\u00fa\u00fb\u00fc\u00ff\u0101\u0105\u0107\u0113\u0119\u011b\u012b\u0131\u0142\u0144\u014d\u0151\u0153\u015b\u016b\u0171\u017a\u017c\u01ce\u01d0\u01d2\u01d4\u0430\u0431\u0432\u0433\u0434\u0435\u0436\u0437\u0438\u0439\u043a\u043b\u043c\u043d\u043e\u043f\u0440\u0441\u0442\u0443\u0444\u0445\u0446\u0447\u0448\u0449\u044a\u044b\u044c\u044d\u044e\u044f\u0451\u0454\u0456\u0457\u0491\u2013!\"'(),-.:;?|~ \u0e01\u0e02\u0e04\u0e06\u0e07\u0e08\u0e09\u0e0a\u0e0b\u0e0c\u0e0d\u0e0e\u0e0f\u0e10\u0e11\u0e12\u0e13\u0e14\u0e15\u0e16\u0e17\u0e18\u0e19\u0e1a\u0e1b\u0e1c\u0e1d\u0e1e\u0e1f\u0e20\u0e21\u0e22\u0e23\u0e24\u0e25\u0e27\u0e28\u0e29\u0e2a\u0e2b\u0e2c\u0e2d\u0e2e\u0e2f\u0e30\u0e31\u0e32\u0e33\u0e34\u0e35\u0e36\u0e37\u0e38\u0e39\u0e40\u0e41\u0e42\u0e43\u0e44\u0e45\u0e46\u0e47\u0e48\u0e49\u0e4a\u0e4b\u0e4c\u0e4d\u2014\u2018\u2019\u201c\u201d",
        punctuations="!\"'(),-.:;?|~ ",
        phonemes="",
        is_unique=True,
        is_sorted=True,
    ),
    phoneme_cache_path=None,
    precompute_num_workers=12,
    start_by_longest=True,
    datasets=DATASETS_CONFIG_LIST,
    cudnn_benchmark=False,
    max_audio_len=SAMPLE_RATE * MAX_AUDIO_LEN_IN_SECONDS,
    mixed_precision=False,
    test_sentences=[
        [
            "ฉันใช้เวลานานมากในการพัฒนาเสียง และตอนนี้ฉันมีมันแล้ว ฉันจะไม่เงียบอีกต่อไป",
            "VCTK_cv005",
            None,
            "th",
        ],
        [
            "ยักษ์ใหญ่ไล่ยักษ์เล็ก ยักษ์เล็กไล่ยักษ์ใหญ่",
            "VCTK_cv068",
            None,
            "th",
        ],
        [
            "ยายกินลำไยน้ำลายยายไหลย้อย",
            "VCTK_cv057",
            None,
            "th",
        ],
        [
            "ชามเขียวคว่ำเช้า ชามขามคว่ำค่ำ",
            "VCTK_cv103",
            None,
            "th",
        ],
        [
            "หมอนลอยน้ำมาว่ายน้ำไปถอยหมอน",
            "VCTK_cv133",
            None,
            "th",
        ],
        [
            "เช้าฟาดผัดฟัก เย็นฟาดฟักผัด",
            "VCTK_cv128",
            None,
            "th",
        ],
    ],
    # # Enable the weighted sampler
    # use_weighted_sampler=True,
    # # Ensures that all speakers are seen in the training batch equally no matter how many samples each speaker has
    # weighted_sampler_attrs={"speaker_name": 1.0},
    # weighted_sampler_multipliers={"Makeitnotblack": None},

    # It defines the Speaker Consistency Loss (SCL) α to 9 like the paper
    speaker_encoder_loss_alpha=9.0,
)

# config.weighted_sampler_multipliers = {}

# print(config.weighted_sampler_multipliers)

# Training

In [ ]:
# Load all the datasets samples and split traning and evaluation sets
train_samples, eval_samples = load_tts_samples(
    config.datasets,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
    formatter=vctk_32k if vctk_config.formatter == "vctk_32k" else None,
)

In [ ]:
# Init the model
model = Vits.init_from_config(config)

In [ ]:
# Init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(restore_path=RESTORE_PATH, skip_train_epoch=SKIP_TRAIN_EPOCH),
    config,
    output_path=OUT_PATH,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples,
)

In [ ]:
trainer.fit()

In [15]:
# weighted_sampler_multipliers={}
# weighted_sampler_multipliers.get(0, None)

In [ ]:
# from dataclasses import dataclass, field
# weighted_sampler_multipliers: dict = field(default_factory=lambda: {})

# print(weighted_sampler_multipliers)

# weighted_sampler_multipliers.get(0, None)

In [ ]:
# @dataclass
# class MyClass:
#     weighted_sampler_multipliers: dict = field(default_factory=lambda: {})

# obj = MyClass()
# print(obj.weighted_sampler_multipliers)